<a href="https://colab.research.google.com/github/jiataoxiang/2048reinforcement_learning/blob/main/2048ReinforcementLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
import torch
import random
import sys
import os
from google.colab import output

In [134]:
# game logic
UP = "w"
DOWN = "s"
LEFT = "a"
RIGHT = "d"
ACTIONS = set([UP, DOWN, LEFT, RIGHT])

def convertAction(action):
    if action == UP:
        return 1
    elif action == DOWN:
        return 3
    elif action == LEFT:
        return 0
    elif action == RIGHT:
        return 2

class Game2048:
    def __init__(self, size):
        self.size = size
        self.board = self.newGame(size)
        self.reward = 0
        self.gameOver = False
    
    def newGame(self, size):
        return torch.zeros((size, size))

    def moveLeft(self, col):
        newCol = torch.zeros((self.size), dtype=self.board.dtype)
        j = 0
        previous = None
        for i in range(self.size):
            if col[i] != 0:
                if previous == None:
                    previous = col[i]
                else:
                    if previous == col[i]:
                        newCol[j] = 2 * col[i]
                        previous = None
                    else:
                        newCol[j] = previous
                        previous = col[i]
                    j += 1
        if previous != None:
            newCol[j] = previous
        return newCol

    def move(self, action):
        action = convertAction(action)
        cur = torch.rot90(self.board, action)
        for i in range(self.size):
            cur[i] = self.moveLeft(cur[i])
        return torch.rot90(cur, -action)

    def addTwo(self):
        indices = (self.board == 0).nonzero()
        if indices.size() != 0:
            randomIndex = random.randint(0, indices.size()[0] - 1)
            self.board[indices[randomIndex][0], indices[randomIndex][1]] = 2
        return self.board

    def checkGameOver(self):
        # if left at least one empty cell
        emptyIndices = (self.board == 0).nonzero()
        if emptyIndices.size()[0] != 0:
            return False
        # check duplicate neighbor
        for i in range(self.size - 1):
            for j in range(self.size - 1):
                if self.board[i, j] == self.board[i + 1, j] or self.board[i, j] == self.board[i, j + 1]:
                    return False
        # Edge case
        for i in range(self.size - 1):
            if self.board[i, 3] == self.board[i + 1, 3]:
                return False
        for j in range(self.size - 1):
            if self.board[3, j] == self.board[3, j + 1]:
                return False
        return True

    def start(self):
        self.addTwo()
        print(self.board)
        while True:
            val = input()
            if val in ACTIONS:
                prev = self.board
                self.board = self.move(val)
                if torch.all(torch.eq(prev, self.board)):
                    print("Invalid move")
                else:
                    over = self.checkGameOver()
                    if over:
                        print("You Lost")
                        return
                    self.addTwo()
                    output.clear()
                    print(self.board)

In [135]:
game = Game2048(4)
game.start()


tensor([[2., 0., 0., 4.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
w
Invalid move


KeyboardInterrupt: ignored